https://gadm.org/index.html<br>
https://hub.arcgis.com/datasets/a21fdb46d23e4ef896f31475217cbb08_1/data?page=2<br>
http://datatopics.worldbank.org/world-development-indicators/<br>
https://datacatalog.worldbank.org/dataset/subnational-population-database<br>
https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/3101.0Sep%202019?OpenDocument<br>
https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html<br>


In [ ]:
# Installing the required libs
!pip install -q fastprogress fastai2 fastcore fast_tabnet --upgrade 

In [ ]:
from fastai2.basics import *
from fastai2.tabular.all import *
from fast_tabnet.core import *

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import sys
sys.path.insert(0, "../input/covid19-global-forecasting-week-2/")
import warnings
warnings.filterwarnings(action='once')

In [ ]:
df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv') 

In [ ]:
df.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv') 
df_test.head()

In [ ]:
df['key'] = df['Country_Region'] + '#' + df['Province_State'].fillna('')
df['province_flag'] = np.where(df['Province_State'].isnull(),0,1)
df['Province_State'] = df['Province_State'].fillna(df['Country_Region'])

In [ ]:
df_test['key'] = df_test['Country_Region'] + '#' + df_test['Province_State'].fillna('')
df_test['province_flag'] = np.where(df_test['Province_State'].isnull(),0,1)
df_test['Province_State'] = df_test['Province_State'].fillna(df_test['Country_Region'])

In [ ]:
df.head(600)

In [ ]:
df_test.head(600)

In [ ]:
#firstconfirmed = df[(df['ConfirmedCases']>0) & (df['Date']<'2020-03-19')].groupby(['Province_State','Country_Region'])['Date'].min().reset_index()
firstconfirmed = df[(df['ConfirmedCases']>0)].groupby(['Province_State','Country_Region'])['Date'].min().reset_index()

In [ ]:
firstconfirmed.head()

In [ ]:
firstconfirmed.shape

In [ ]:
firstfatality = df[(df['Fatalities']>0)].groupby(['Province_State','Country_Region'])['Date'].min().reset_index()
#firstfatality = df[(df['Fatalities']>0) & (df['Date']<'2020-03-19')].groupby(['Province_State','Country_Region'])['Date'].min().reset_index()

In [ ]:
firstfatality.head()

In [ ]:
firstfatality.shape

In [ ]:
firstconfirmed.columns = ['Province_State','Country_Region','FirstCaseDate']
firstfatality.columns = ['Province_State','Country_Region','FirstFatalityDate']

In [ ]:
df = df.merge(firstconfirmed, left_on=['Province_State','Country_Region'],right_on=['Province_State','Country_Region'],how='left')
df = df.merge(firstfatality, left_on=['Province_State','Country_Region'],right_on=['Province_State','Country_Region'],how='left')

In [ ]:
df.head(40)

In [ ]:
df_test = df_test.merge(firstconfirmed, left_on=['Province_State','Country_Region'],right_on=['Province_State','Country_Region'],how='left')
df_test = df_test.merge(firstfatality, left_on=['Province_State','Country_Region'],right_on=['Province_State','Country_Region'],how='left')

In [ ]:
df_test.head(40)

In [ ]:
df.dtypes

In [ ]:
df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True) 
df['FirstCaseDate']=pd.to_datetime(df['FirstCaseDate'], infer_datetime_format=True) 
df['FirstFatalityDate']=pd.to_datetime(df['FirstFatalityDate'], infer_datetime_format=True) 

In [ ]:
df.head()

In [ ]:
df_test['Date']=pd.to_datetime(df_test['Date'], infer_datetime_format=True) 
df_test['FirstCaseDate']=pd.to_datetime(df_test['FirstCaseDate'], infer_datetime_format=True) 
df_test['FirstFatalityDate']=pd.to_datetime(df_test['FirstFatalityDate'], infer_datetime_format=True) 

In [ ]:
df_test.head()

In [ ]:
df['days_first_case']=(df['Date']-df['FirstCaseDate']).dt.days
df['days_first_fatality']=(df['Date']-df['FirstFatalityDate']).dt.days

In [ ]:
df['days_first_case']

In [ ]:
df_test['days_first_case']=(df_test['Date']-df_test['FirstCaseDate']).dt.days
df_test['days_first_fatality']=(df_test['Date']-df_test['FirstFatalityDate']).dt.days

In [ ]:
df_test['days_first_case']

In [ ]:
df['days_first_case']=np.where(df['days_first_case']<0,0,df['days_first_case'].fillna(0))
df['days_first_fatality']=np.where(df['days_first_fatality']<0,0,df['days_first_fatality'].fillna(0))

In [ ]:
df_test['days_first_case']=np.where(df_test['days_first_case']<0,0,df_test['days_first_case'].fillna(0))
df_test['days_first_fatality']=np.where(df_test['days_first_fatality']<0,0,df_test['days_first_fatality'].fillna(0))

In [ ]:
df.tail()

In [ ]:
df_test.tail()

In [ ]:
df[df['Country_Region']=='Brazil'].tail()

In [ ]:
df_test[df_test['Country_Region']=='Brazil'].tail()

In [ ]:
add_datepart(df,'Date',drop=False)

In [ ]:
add_datepart(df_test,'Date',drop=False)

In [ ]:
external = pd.read_csv('/kaggle/input/covid19-week2-external-data/external_data.csv',sep=';',decimal=',')

In [ ]:
external.head()

In [ ]:
df = df.merge(external, left_on='key',right_on='key',how='left')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df_test = df_test.merge(external, left_on='key',right_on='key',how='left')

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
list(df)

In [ ]:
df.pivot_table(index='Country_Region', columns='Date', values='ConfirmedCases', aggfunc=np.sum, fill_value=0)

In [ ]:
df['Confirmedlast43'] = df['ConfirmedCases'].shift(43) 
df['Fatalitieslast43'] = df['Fatalities'].shift(43)

In [ ]:
df['is_valid'] = np.where(df['Date']<'2020-03-29', False, True)

In [ ]:
df.groupby('is_valid').size()

In [ ]:
df['ConfirmedLog'] = np.log(df['ConfirmedCases']+1)
df['FatalitiesLog'] = np.log(df['Fatalities']+1)

In [ ]:
cat_vars = ['Province_State','Country_Region','province_flag']
cont_vars = ['Elapsed',
             'days_first_case',
             'days_first_fatality',
             'pop_density',
             'population',
             'area',
             'lat_min',
             'lat_max',
             'lon_min',
             'lon_max',
             'centroid_x',
             'centroid_y',
             'wdi_country_population',
             'wdi_country_arrivals',
             'wdi_arrivals_per_capita',
             'wdi_gini',
             'wdi_perc_urban_pop',
             'wdi_perc_handwashing',
             'wdi_uhc_coverage',
             'wdi_hospital_beds_p1000',
             'wdi_smoke_prevalence',
             'wdi_diabetes_prevalence',
             'wdi_gdp_per_capita_ppp',
             'wdi_perc_death_comm_diseases',
             'wdi_perc_death_non_comm_diseases',
             'wdi_death_rate_p1000',
             'wdi_perc_basic_sanitation',
             'wdi_dom_govmt_healt_exped_gdp',
             'wdi_dom_govmt_healt_exped_per_cap',
             'wdi_perc_females',
             'wdi_perc_males',
             'wdi_perc_female_20_29',
             'wdi_perc_female_30_39',
             'wdi_perc_female_40_49',
             'wdi_perc_female_50_59',
             'wdi_perc_female_60_69',
             'wdi_perc_female_70_79',
             'wdi_perc_female_80p',
             'wdi_perc_male_20_29',
             'wdi_perc_male_30_39',
             'wdi_perc_male_40_49',
             'wdi_perc_male_50_59',
             'wdi_perc_male_60_69',
             'wdi_perc_male_70_79',
             'wdi_perc_male_80p',
             'wdi_pop_denisty',
             'wdi_perc_anual_growth_pop','Month','Week','Dayofyear']#,'Confirmedlast43','Fatalitieslast43']
dep_vars = ['ConfirmedLog']#,'Fatalities']

In [ ]:
procs = [FillMissing, Categorify, Normalize]
splits = ColSplitter('is_valid')(df)

In [ ]:
splits

In [ ]:
#?TabularPandas

In [ ]:
y_block = TransformBlock()

In [ ]:
to = TabularPandas(df, procs, cat_names=cat_vars, cont_names=cont_vars, y_names=dep_vars, y_block=RegressionBlock(), splits=splits)

In [ ]:
to

In [ ]:
dls = to.dataloaders(bs=512)

In [ ]:
dls.show_batch()

In [ ]:
cats, conts, y = next(iter(dls.train))

In [ ]:
cats.shape

In [ ]:
conts.shape

In [ ]:
y.shape

In [ ]:
dls.c

In [ ]:
#max_log_y = np.log(10) + np.max(df['ConfirmedCases'])
max_log_y = np.log(1.2) + np.max(df['ConfirmedLog'])
y_range = (0, max_log_y)
y_range

In [ ]:
learn = tabular_learner(dls, layers=[300,250], loss_func=MSELossFlat(),
                        config=tabular_config(ps=[0.005,0.015]), y_range=y_range, 
                        metrics=[exp_rmspe, rmse])#, embed_p=0.02

In [ ]:
learn.model

In [ ]:
learn.load('/kaggle/input/covid19-week2-external-data/best18_2day')

In [ ]:
preds, y = learn.get_preds()

In [ ]:
np.exp(preds)-1

In [ ]:
np.exp(y)-1

In [ ]:
dl = learn.dls.test_dl(df)

In [ ]:
raw_test_preds = learn.get_preds(dl=dl)

In [ ]:
raw_test_preds[0]

In [ ]:
raw_test_preds[1]

In [ ]:
preds = np.exp(to_np(raw_test_preds[0]))-1

In [ ]:
preds[:40]

In [ ]:
df['ConfirmedCases'].head(40)

In [ ]:
df.head(40)

In [ ]:
preds = pd.DataFrame(preds)
preds.columns = ['pred_confirmed']

In [ ]:
preds.tail()

In [ ]:
df = pd.concat([df, preds], axis=1)

In [ ]:
df.tail()

## Infer Test Confirmed

In [ ]:
df_test

In [ ]:
dl = learn.dls.test_dl(df_test)

In [ ]:
raw_test_preds = learn.get_preds(dl=dl)

In [ ]:
raw_test_preds[0]

In [ ]:
preds_confirmed_test = np.exp(to_np(raw_test_preds[0]))-1

In [ ]:
preds_confirmed_test[:40]

## Fatalities Model

In [ ]:
cat_vars = ['Province_State','Country_Region','province_flag']
cont_vars = ['Elapsed',
             'days_first_case',
             'days_first_fatality',
             'pop_density',
             'population',
             'area',
             'lat_min',
             'lat_max',
             'lon_min',
             'lon_max',
             'centroid_x',
             'centroid_y',
             'wdi_country_population',
             'wdi_country_arrivals',
             'wdi_arrivals_per_capita',
             'wdi_gini',
             'wdi_perc_urban_pop',
             'wdi_perc_handwashing',
             'wdi_uhc_coverage',
             'wdi_hospital_beds_p1000',
             'wdi_smoke_prevalence',
             'wdi_diabetes_prevalence',
             'wdi_gdp_per_capita_ppp',
             'wdi_perc_death_comm_diseases',
             'wdi_perc_death_non_comm_diseases',
             'wdi_death_rate_p1000',
             'wdi_perc_basic_sanitation',
             'wdi_dom_govmt_healt_exped_gdp',
             'wdi_dom_govmt_healt_exped_per_cap',
             'wdi_perc_females',
             'wdi_perc_males',
             'wdi_perc_female_20_29',
             'wdi_perc_female_30_39',
             'wdi_perc_female_40_49',
             'wdi_perc_female_50_59',
             'wdi_perc_female_60_69',
             'wdi_perc_female_70_79',
             'wdi_perc_female_80p',
             'wdi_perc_male_20_29',
             'wdi_perc_male_30_39',
             'wdi_perc_male_40_49',
             'wdi_perc_male_50_59',
             'wdi_perc_male_60_69',
             'wdi_perc_male_70_79',
             'wdi_perc_male_80p',
             'wdi_pop_denisty',
             'wdi_perc_anual_growth_pop','Month','Week','Dayofyear','pred_confirmed']
dep_vars = ['FatalitiesLog'] 

In [ ]:
procs = [FillMissing, Categorify, Normalize]
splits = ColSplitter('is_valid')(df)

In [ ]:
to = TabularPandas(df, procs, cat_names=cat_vars, cont_names=cont_vars, y_names=dep_vars, y_block=RegressionBlock(), splits=splits)

In [ ]:
dls = to.dataloaders(bs=512)

In [ ]:
max_log_y = np.log(1.2) + np.max(df['FatalitiesLog'])
y_range = (0, max_log_y)
y_range

In [ ]:
learn = tabular_learner(dls, layers=[300,250], loss_func=MSELossFlat(),
                        config=tabular_config(ps=[0.005,0.015]), y_range=y_range, 
                        metrics=[exp_rmspe, rmse])#, embed_p=0.02

In [ ]:
learn.load('/kaggle/input/covid19-week2-external-data/fat_best2_2day')

In [ ]:
preds, y = learn.get_preds()

In [ ]:
preds

In [ ]:
y

In [ ]:
preds_confirmed_test = pd.DataFrame(preds_confirmed_test)

In [ ]:
preds_confirmed_test.columns=['ConfirmedCases']

In [ ]:
preds_confirmed_test.tail()

In [ ]:
preds_confirmed_test.shape

In [ ]:
df_test.shape

In [ ]:
df_test = pd.concat([df_test, preds_confirmed_test], axis=1)

In [ ]:
df_test.tail(40)

In [ ]:
df_test['pred_confirmed']=df_test['ConfirmedCases']

In [ ]:
dl = learn.dls.test_dl(df_test)

In [ ]:
raw_test_preds = learn.get_preds(dl=dl)

In [ ]:
raw_test_preds[0]

In [ ]:
preds_fatalities_test = np.exp(to_np(raw_test_preds[0]))-1

In [ ]:
preds_fatalities_test

In [ ]:
preds_fatalities_test = pd.DataFrame(preds_fatalities_test)

In [ ]:
preds_fatalities_test.columns=['Fatalities']

In [ ]:
preds_fatalities_test.tail()

In [ ]:
preds_fatalities_test.shape

In [ ]:
df_test = pd.concat([df_test, preds_fatalities_test], axis=1)

In [ ]:
df_test[df_test['Country_Region']=='Brazil'].tail()

In [ ]:
df_test[df_test['Country_Region']=='Brazil'].head(20)

In [ ]:
sub_ex = df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/submission.csv')  

In [ ]:
sub_ex.head()

In [ ]:
sub = df_test[['ForecastId','ConfirmedCases','Fatalities']]

In [ ]:
sub.head()

In [ ]:
sub_ex.tail()

In [ ]:
sub.tail()

In [ ]:
sub.to_csv('submission.csv',index=False)